In [1]:
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude
from tensorflow_model_optimization.sparsity.keras import ConstantSparsity, PolynomialDecay
from tensorflow_model_optimization.sparsity.keras import UpdatePruningStep

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from LTH_helper import LTH
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import initializers

In [23]:
def compile_model(model, lr=0.001):
    optim = optimizers.Adam(lr=lr)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    
def get_callbacks(filename):
    return [
        ReduceLROnPlateau(monitor='val_accuracy', mode='max', factor=np.sqrt(0.1), patience=10, verbose=1, min_lr=1e-4),
        ModelCheckpoint(filepath=filename,  verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')
    ]

def get_prunned_callbacks(filename):
    return get_callbacks(filename) + [UpdatePruningStep()]

In [74]:
sprasity_sched = ConstantSparsity(
            0.99, 
            10*(len(X_train)//batch_size), # desde que step arranca a calcular la mascara
            end_step=-1, # Hasta cuando recalcula la mascara
            frequency=(len(X_train)//batch_size) # Cada cuantos steps recalcula la mascara
        )

model = Sequential()
model.add(prune_low_magnitude(Dense(784*2, activation='relu', name='hidden_1', input_dim=784), sprasity_sched))
model.add(prune_low_magnitude(Dense(784, activation='relu', name='hidden_2'), sprasity_sched))
model.add(prune_low_magnitude(Dense(10, activation='softmax', name='Salida'), sprasity_sched))

In [75]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
X = train_images.reshape(-1, 784)/255
y = train_labels.reshape(-1, 1)
X_test = test_images.reshape(-1, 784)/255
y_test = test_labels.reshape(-1, 1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [76]:
epochs = 20
batch_size = 256
compile_model(model)
history = model.fit(X_train, 
               y_train,
               epochs=epochs, batch_size=batch_size, 
              validation_data = (X_val, y_val),
               verbose=2, 
                    shuffle = True,
                           callbacks=get_prunned_callbacks('mlp.mnist.sparse_train_testing.hdf5')
              )

Epoch 1/20

Epoch 00001: val_accuracy improved from -inf to 0.85489, saving model to mlp.mnist.sparse_train_testing.hdf5
200/200 - 1s - loss: 0.5055 - accuracy: 0.8184 - val_loss: 0.3935 - val_accuracy: 0.8549 - lr: 0.0010
Epoch 2/20

Epoch 00002: val_accuracy improved from 0.85489 to 0.86744, saving model to mlp.mnist.sparse_train_testing.hdf5
200/200 - 1s - loss: 0.3551 - accuracy: 0.8710 - val_loss: 0.3709 - val_accuracy: 0.8674 - lr: 0.0010
Epoch 3/20

Epoch 00003: val_accuracy did not improve from 0.86744
200/200 - 1s - loss: 0.3159 - accuracy: 0.8833 - val_loss: 0.3653 - val_accuracy: 0.8636 - lr: 0.0010
Epoch 4/20

Epoch 00004: val_accuracy improved from 0.86744 to 0.87933, saving model to mlp.mnist.sparse_train_testing.hdf5
200/200 - 1s - loss: 0.2938 - accuracy: 0.8903 - val_loss: 0.3374 - val_accuracy: 0.8793 - lr: 0.0010
Epoch 5/20

Epoch 00005: val_accuracy improved from 0.87933 to 0.88289, saving model to mlp.mnist.sparse_train_testing.hdf5
200/200 - 1s - loss: 0.2735 - ac

In [77]:
layer = 0
(model.layers[layer].pruning_vars[0][0] == 0).numpy().sum()/np.product(model.layers[layer].pruning_vars[0][0].numpy().shape)

0.9900000976155768